<a href="https://colab.research.google.com/github/NamanJhawar-NMIMS/Project-Search-Engine/blob/main/Project_Search_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IMPORTING LIBRARIES 

In [1]:
pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=8de62f2002ec7a20833318313d1857ce7649b367ee9e5da45d6136ce87dee6ac
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [2]:
import pandas as pd
import numpy as np

import re 

import wget

import gensim.models

#Library to tokenise and remove stopwords 
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
try:
    nlp=spacy.load('en_core_web_sm')
except:
    os.system('python -m spacy download en_core_web_sm')
    nlp=spacy.load('en_core_web_sm')


#Library to remove stopwords 
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
data=pd.read_csv(wget.download("https://raw.githubusercontent.com/NamanJhawar-NMIMS/Project-Search-Engine/main/project_description.csv","project_description.csv"))
data.head(5)

,project_id,title,description
0,1,Walmart Sales Forecasting Data Science Project,Every Departmental store chain like Walmart wa...
1,2,Credit Card Default Prediction using Machine l...,Business Context\nBanks are primarily known fo...
2,3,Analyse Yelp Dataset with Spark & Parquet Form...,Business Overview\nAzure Databricks is a data ...
3,4,Expedia Hotel Recommendations Data Science Pro...,"Planning your dream vacation, or even a weeken..."
4,5,Data Science Project in Python on BigMart Sale...,"Occasionally, we all love to go on a shopping ..."


In [4]:
data.count()

project_id     205
title          205
description    205
dtype: int64

## PREPROCESSING THE DATA

In [5]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__) 

In [6]:
# defining functions for different pre processing methods
def separate_words_if_capital(data):
    try:
        separated_data=re.sub(r"(?<=[a-z])(?=[A-Z])"," ", data)
        # to introduce space between all capitalised letters
    except Exception as e:
            logger.exception(e)
            print("Words are still not separated with space")
    else:
        return separated_data
    
def lower_case(data):
    try:
        lower_data=data.lower()   # converting the text to lower case
    except Exception as e:
            logger.exception(e)
            print("Data not converted to lower case")
    else:
        return lower_data
#string.punctuation contains : #!”#$%&\()*+,-/:;<=>?@[\\]^_`{|}~
def remove_punctuations(data):
    try:
        trans = str.maketrans('','', string.punctuation)
        transformed_data=data.translate(trans)  # removing all the defined punctuations from the text
    except Exception as e:
            logger.exception(e)
            print("Punctuations not removed")
    else:
        return transformed_data
def white_space(data):
    try:
        space_removed_data=' '.join(data.split())   # removing white spaces from text
    except Exception as e:
            logger.exception(e)
            print("White spaces were not removed")
    else:
        return space_removed_data
    
def remove_non_textual_data(data):
    try:
        regex_compiled=re.compile(r'(?:|:|;|&|\(|\)|\?|-|!|\n|/|,|_|"|"|\n+|\t|\t+|•|●|:black_circle:|✓|https\S|e.g.|nbsp|\'|’|➔|\#|@|\+|\*|\?|\[|\^|\]|\$|\(|\)|\{|\}|\=|\!|\-|~|`|%|\|\<|\>)')
        text_data=re.sub(regex_compiled,'',data)   # removing links, bullet points, tick marks, non text data
    except Exception as e:
            logger.exception(e)
            print("Non-textual data is still present")
    else:
        return text_data   
        
def remove_numbers(data):
# Iterate over the characters in string and join all characters except digits, to create a new string.
    try:
            mod_string = ''.join((item for item in data if not item.isdigit()))
    except Exception as e:
            logger.exception(e)
            print("Numbers from the data were not removed")
    else:
            return mod_string
# function for tokenization of pre processed text
def tokenizer(text):
    try:
            tokenised_text=[]
            for token in nlp(text):
                tokenised_text.append(token)
    except Exception as e:
            logger.exception(e)
            print("tokenization is not completed")
    else:
            return tokenised_text
# function to remove stop words using list comprhension and returns a string
def remove_stopwords(text):
    try:
            #nltk and spacy combined stopwords
            stpwrds=set(stopwords.words('english')).union(STOP_WORDS)
            text_cleaned=[token for token in text if not (str(token).lower() in stpwrds)]
            listToStr = ' '.join([str(elem) for elem in text_cleaned])
    except Exception as e:
            logger.exception(e)
            print("Stopwords not removed")
    else:
            return listToStr
# function to lemmatize and find root words
def extract_lemma(text_cleaned):
    try:
            text_lemma=[]
            text_cleaned=nlp(text_cleaned)
            for token in text_cleaned:
                text_lemma.append(token.lemma_)
            listToStr = ' '.join([str(elem) for elem in text_lemma])
    except Exception as e:
            logger.exception(e)
            print("lemmatization not completed")
    else:
            return listToStr
# main function for pre processing the data : returns final pre processed text in processed_data
def sub_preprocessing(data):
    try:
            separated_data=separate_words_if_capital(data)
            data_lower=lower_case(separated_data)
            space_removed_data = white_space(data_lower)
            text_data = remove_non_textual_data(space_removed_data)
            processed_data=remove_numbers(text_data)
    except Exception as e:
            logger.exception(e)
    else:
            return processed_data
# calling all functions to pre process , cleaning the text and convert to dataframe
def final_preprocessing(df,content_col):
    try:
            df['Processed_text']=df.apply(lambda x: sub_preprocessing(x[content_col]), axis=1)
            df['Processed_text']=df.apply(lambda x: tokenizer(x['Processed_text']), axis=1)
            df['Processed_text']=df.apply(lambda x: remove_stopwords(x['Processed_text']), axis=1)
            df['Processed_text']=df.apply(lambda x: extract_lemma(x['Processed_text']), axis=1)
            df.project_id=pd.to_numeric(df.project_id)
    except Exception as e:
            logger.exception(e)
    else:
        return df

In [7]:
preprocessed_data=final_preprocessing(data,'description')
preprocessed_data.head(5)

,project_id,title,description,Processed_text
0,1,Walmart Sales Forecasting Data Science Project,Every Departmental store chain like Walmart wa...,departmental store chain like walmart wants pr...
1,2,Credit Card Default Prediction using Machine l...,Business Context\nBanks are primarily known fo...,business context bank primarily know money len...
2,3,Analyse Yelp Dataset with Spark & Parquet Form...,Business Overview\nAzure Databricks is a data ...,business overview azure databrick data analyti...
3,4,Expedia Hotel Recommendations Data Science Pro...,"Planning your dream vacation, or even a weeken...",planning dream vacation weekend escape overwhe...
4,5,Data Science Project in Python on BigMart Sale...,"Occasionally, we all love to go on a shopping ...",occasionally love shopping spree realize later...


In [8]:
data

,project_id,title,description,Processed_text
0,1,Walmart Sales Forecasting Data Science Project,Every Departmental store chain like Walmart wa...,departmental store chain like walmart wants pr...
1,2,Credit Card Default Prediction using Machine l...,Business Context\nBanks are primarily known fo...,business context bank primarily know money len...
2,3,Analyse Yelp Dataset with Spark & Parquet Form...,Business Overview\nAzure Databricks is a data ...,business overview azure databrick data analyti...
3,4,Expedia Hotel Recommendations Data Science Pro...,"Planning your dream vacation, or even a weeken...",planning dream vacation weekend escape overwhe...
4,5,Data Science Project in Python on BigMart Sale...,"Occasionally, we all love to go on a shopping ...",occasionally love shopping spree realize later...
...,...,...,...,...
200,201,Learn to Build a Neural network from Scratch u...,Overview\nA neural network is a set of algorit...,overview neural network set algorithm aim reco...
201,202,PySpark Project-Build a Data Pipeline using Hi...,Agenda\nThis is the seventh project in the Pys...,agenda seventh project pyspark series . sixth ...
202,203,PySpark Project-Build a Data Pipeline using Ka...,Agenda\nThis is the eighth project in the Pysp...,agenda th project pyspark series . seventh pro...
203,204,NLP Project for Multi Class Text Classificatio...,"Business Overview&nbsp;\nSo far, in this serie...",business overview far series nlp project multi...


In [104]:
#Creating sentences from the processed text 

sentences=[]

keyed_sentence={}
for i in data.index:
  sentences.append(data['Processed_text'][i].split("."))
  #print(i, len(data['Processed_text'][i].split(".")[0].strip()),data['title'][i])
  keyed_sentence[data['Processed_text'][i].split(".")[0].strip()]=data['title'][i]



## Doc2Vec

In [12]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [13]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(data['Processed_text'])]
model = Doc2Vec(documents, vector_size=5, window=15, min_count=1, workers=4)

In [14]:
vectors=[]
for i in range(0,len(sentences)):
    vector=model.docvecs[i]
    vectors.append(vector)
len(vectors)

205

In [15]:
vector = model.infer_vector(['neural','network','projects','in','R'])
print(vector)

[-0.01878526 -0.07591928  0.02124105  0.09838784 -0.00238787]


In [16]:
inputSentence='Tensorflow projects in python'


In [18]:
def output_sentences(most_similar):
    print('\n')
    for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(most_similar)//2), ('LEAST', len(most_similar) - 1)]:
      print(u'%s %s: %s\n' % (label, most_similar[index][1], list(data[data.project_id==most_similar[index][0]].title)[0]))
      print('=====================\n')

vector = model.infer_vector(inputSentence.split(" "))
most_similar = model.docvecs.most_similar([vector]) 
output_sentences(most_similar)



MOST 0.3800223469734192: Retail Analytics Project Example using Sqoop, HDFS, and Hive


SECOND-MOST 0.31028780341148376: Topic modelling using Kmeans clustering to group customer reviews


MEDIAN 0.22782866656780243: Forecasting Business KPI's with Tensorflow and Python


LEAST 0.13061702251434326: PySpark Project to Learn Advanced DataFrame Concepts




https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html#sphx-glr-auto-examples-tutorials-run-doc2vec-lee-py 

## Glove 

In [19]:
#!pip install glove-python-binary

In [20]:
# importing the glove library
# from glove import Corpus, Glove

In [21]:
# # creating a corpus object
# corpus = Corpus() 
# #training the corpus to generate the co occurence matrix which is used in GloVe
# corpus.fit(sentences, window=10)
# #creating a Glove object which will use the matrix created in the above lines to create embeddings
# #We can set the learning rate as it uses Gradient Descent and number of components
# glove = Glove(no_components=5, learning_rate=0.05)
 
# glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
# glove.add_dictionary(corpus.dictionary)

In [22]:
# print(glove.word_vectors[glove.dictionary['neural']])

## S-BERT

In [23]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.7 MB/s 
     |████████████████████████████████| 4.7 MB 26.3 MB/s 
     |████████████████████████████████| 1.3 MB 54.4 MB/s 
     |████████████████████████████████| 120 kB 62.8 MB/s 
     |████████████████████████████████| 6.6 MB 41.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=8f3fb020647c9d539956b6ce226a02b6182658cb6e702da65fd53f2c2b032c17
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [24]:
from sentence_transformers import SentenceTransformer  # importing sentence transformer
sbert_model= SentenceTransformer('all-MiniLM-L6-v2') # using pre trained model for Sbert
sentence_embeddings=sbert_model.encode(sentences)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [25]:
sentence_embeddings[1].shape


(384,)

In [26]:
sentence_embeddings.shape

(205, 384)

In [27]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

In [28]:
test_data=[['neural','network','projects','in','R']]

In [29]:
from sentence_transformers.util import cos_sim

similarity = []
sim_df=pd.DataFrame(columns=['title','similarity'])
df_title=[]
df_similarity=[]
for sent in sentences:
  sim = 1-cosine(sbert_model.encode(test_data), sbert_model.encode([sent])[0])
  #print("Sentence = ", sent, "; similarity = ", sim)
  df_title.append(keyed_sentence[sent[0].strip()])
  df_similarity.append(sim)

sim_df.title=df_title
sim_df.similarity=df_similarity






In [30]:

sim_df.sort_values(ascending=False,by='similarity').head(5)

,title,similarity
111,Deep Learning Project for Beginners with Sourc...,0.397482
192,PyTorch Project to Build a LSTM Text Classific...,0.396585
69,Medical Image Segmentation Deep Learning Project,0.393463
200,Learn to Build a Neural network from Scratch u...,0.373576
185,Build a CNN Model with PyTorch for Image Class...,0.369544


## InferSent

In [31]:
! mkdir encoder
! curl -Lo encoder/infersent2.pkl https://dl.fbaipublicfiles.com/infersent/infersent2.pkl
  
! mkdir GloVe
! curl -Lo GloVe/glove.840B.300d.zip http://nlp.stanford.edu/data/glove.840B.300d.zip
! unzip GloVe/glove.840B.300d.zip -d GloVe/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  146M  100  146M    0     0  8206k      0  0:00:18  0:00:18 --:--:-- 8952k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0   315    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0   353    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 2075M  100 2075M    0     0  5173k      0  0:06:50  0:06:50 --:--:-- 5108k
Archive:  GloVe/glove.840B.300d.zip
  inflating: GloVe/glove.840B.300d.txt  


In [33]:
pip install models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement models (from versions: 0.0.3, 0.0.4, 0.0.5, 0.1.0, 0.1.1, 0.2.0, 0.3.0, 0.4.0, 0.7.0, 0.8.0, 0.9.0, 0.9.1, 0.9.2, 0.9.3)
ERROR: No matching distribution found for models


In [45]:
modelspy=wget.download("https://raw.githubusercontent.com/facebookresearch/InferSent/main/models.py","models.py")


In [46]:
from models import InferSent
import torch

V = 2
MODEL_PATH = 'encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

W2V_PATH = '/content/GloVe/glove.840B.300d.txt'
model.set_w2v_path(W2V_PATH)

In [158]:
sentences_infersent=[]
keyed_sentence_infersent={}
for i in data.index:
  sentences_infersent.append(data['Processed_text'][i].replace(".",""))
  #print(i, len(data['Processed_text'][i].split(".")[0].strip()),data['title'][i])
  keyed_sentence_infersent[data['title'][i]]=data['Processed_text'][i].replace(".","")


model.build_vocab(sentences_infersent, tokenize=True)

Found 3786(/4733) words with w2v vectors
Vocab size : 3786


In [160]:
query = "Deep Learning using R"
query_vec = model.encode(query)[0]
print(query_vec)
print(query_vec.shape)


[-0.01715463  0.04943122 -0.17540526 ... -0.01258786 -0.06795799
 -0.04156346]
(4096,)


/content/models.py:207: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentences = np.array(sentences)[idx_sort]


In [162]:

similarity = []
sim_infersent_df=pd.DataFrame(columns=['title','similarity'])
df_title_infersent=[]
df_similarity_infersent=[]

for sent in sentences_infersent:
  sim = 1-cosine(query_vec, model.encode([sent])[0])
  #print(sim, sent)
  for i in range(len(keyed_sentence_infersent)):
    if sent in list(keyed_sentence_infersent.values()):
      title_infersent=list(keyed_sentence_infersent.keys())[list(keyed_sentence_infersent.values()).index(sent)]
      print(sim,title_infersent)
      break
    else:
      print("Not found")
      #print("Sentence = ", sent, "; similarity = ", sim)
  df_title_infersent.append(title_infersent)
  df_similarity_infersent.append(sim)

sim_infersent_df.title=df_title_infersent
sim_infersent_df.similarity=df_similarity_infersent



-0.11722125113010406 Walmart Sales Forecasting Data Science Project
-0.11822658032178879 Credit Card Default Prediction using Machine learning techniques
-0.11807144433259964 Analyse Yelp Dataset with Spark & Parquet Format on Azure Databricks
-0.13671371340751648 Expedia Hotel Recommendations Data Science Project
-0.15535998344421387 Data Science Project in Python on BigMart Sales Prediction
-0.15861184895038605 Inventory Demand Forecasting using Machine Learning in R
-0.1047697588801384 Machine Learning Project to Forecast Rossmann Store Sales
-0.11179642379283905 Build an Azure Recommendation Engine on Movielens Dataset
-0.11950532346963882 Bitcoin Data Mining on AWS Free Tier
-0.11723306775093079 Ensemble Machine Learning Project - All State Insurance Claims Severity Prediction
-0.11628231406211853 Web Server Log Processing using Hadoop in Azure
-0.11754697561264038 Hadoop Project-Analysis of Yelp Dataset using Hadoop Hive
-0.10439769178628922 Predict Macro Economic Trends using Ka

In [165]:
sim_infersent_df.sort_values(ascending=False,by="similarity").head(5)

,title,similarity
42,German Credit Dataset Analysis to Classify Loa...,-0.034433
50,Identifying Product Bundles from Sales Data Us...,-0.051663
131,Azure Text Analytics for Medical Search Engine...,-0.062982
55,Event Data Analysis using AWS ELK Stack,-0.068334
51,Ecommerce product reviews - Pairwise ranking a...,-0.070854
